# Script de Raspagem e Download dos dados

Matheus Costa e Silva - 24 de outubro de 2023

## Preparação de ambiente

In [ ]:
import os
import sys
import re
import zipfile
import tempfile
import io
import requests
from functools import reduce
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import StructType,StructField, StringType
from bs4 import BeautifulSoup

#Create SparkSession
spark = SparkSession.builder.appName('Daiichi').getOrCreate()


### Funções Auxiliares

In [ ]:


def baixar_descompactar_e_salvar_csv_com_pyspark(zip_file_url, zip_file_name, csv_file_name,
                                                 endereco, area,UF, ANO):
    with tempfile.TemporaryDirectory() as temp_dir:""
        # Baixar o arquivo ZIP
        response = requests.get(zip_file_url)
        with open(f"{temp_dir}/{zip_file_name}", "wb") as zip_file:
            zip_file.write(response.content)

        # Descompactar o arquivo ZIP
        with zipfile.ZipFile(f"{temp_dir}/{zip_file_name}", "r") as zip_ref:
            zip_ref.extractall(temp_dir)

        # Inicializar o SparkSession
        spark = SparkSession.builder.appName("Leitura_CSV_Com_PySpark").getOrCreate()

        # Ler o CSV com o PySpark
        csv_file_path = f"{temp_dir}/{csv_file_name}"
        df = spark.read.options(delimiter=';').csv(csv_file_path, header=True, inferSchema=True)

        # Realizar operações com o DataFrame (opcional)
        #df.show()
        df.write.format("csv").option("header",True).mode('append').save(str(UF)+str(ANO)+str(area))

        # Encerrar a sessão do Spark
        #spark.stop()



def processar_salvar_estado_base(endereco, area,UF, ANO):
    
    url = str(endereco)+"/"+str(ANO)+"/"+str(UF)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True)]


    r = re.compile(".*"+area)
    aux= list(filter(r.match, links))
    r2 = re.compile(".*12")
    newlist=list(filter(r2.match,aux))
    print(newlist)
    for zip_url in newlist:
        if zip_url.endswith(area+".zip"):
            zip_file_name = zip_url
            csv_file_name = zip_file_name.replace(".zip", ".csv")
            print(url+"/"+zip_url)
            print(zip_file_name)
            print(csv_file_name)
        #print(csv_file_name)
            baixar_descompactar_e_salvar_csv_com_pyspark(url+"/"+zip_url, zip_file_name, csv_file_name,
                                                 endereco, area,UF, ANO)
            
    
def analise_anual_full(URL,ano):
    url = str(URL)+"/"+str(ano)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True)]
    areas = ['CONS','DET']
    for area in areas:
        for uf in links:
            processar_salvar_estado_base(URL,area,uf,str(ano))

def amb_full(URL):
    url = str(URL)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [a['href'] for a in soup.find_all('a', href=True)]
    r = re.compile(".*202")
    aux= list(filter(r.match, links))
    for ano in aux:
        print(URL)
        print(ano)
        analise_anual_full(URL,ano)
            

## Download de toda base de dados TISS

### Para fins de otimização de computação e armazenamento, optou-se por baixar somente a competência de dezembro dos anos de 2020, 2021 e 2022 para os seguintes estados: Goiás, Bahia e Minas Gerais

In [ ]:
amb_full("https://dadosabertos.ans.gov.br/FTP/PDA/TISS/AMBULATORIAL")

## Download de toda a base de dados de planos de saúde cadastrados

In [ ]:
baixar_descompactar_e_salvar_csv_com_pyspark("https://dadosabertos.ans.gov.br/FTP/PDA/TISS/DADOS_DE_PLANOS/PLANOS.zip",
                                             "PLANOS.zip", "PLANOS.csv",
                                                 "endereco", "TOTAL","BR", "FULL")

df = spark.read.options(delimiter=',').csv("BRFULLTOTAL", header=True, inferSchema=True)

## Download da base de dados de beneficiários cadastrados nos meses e anos discutidos acima para as mesmas UF's

In [77]:
from pyspark.sql.functions import input_file_name
import os

# Inicie uma sessão do Spark
spark = SparkSession.builder.appName("Leitura de CSV").getOrCreate()

# Diretório raiz onde estão as pastas das UF's
URL = "https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios"

# Anos que você deseja processar
anos = ["202012","202112","202212"]

# Lista para armazenar os DataFrames de todas as UF's
dfs_ufs = []

# Loop através das UF's
#aux = os.listdir(diretorio_raiz)
ufs = ["GO","MG","BA"]
for ano in anos:
    for uf in ufs:
        url = str(URL)+"/"+str(ano)
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [a['href'] for a in soup.find_all('a', href=True)]
        #ufs = ["GO","MG","BA"]
        r = re.compile(".*"+uf)
        aux= list(filter(r.match, links))
        print(aux)
        for zip_file_name in aux:
            print(zip_file_name)
            print(zip_file_name.replace(".zip", ".csv"))
            print(url+"/"+zip_file_name)
            baixar_descompactar_e_salvar_csv_com_pyspark(url+"/"+zip_file_name,
                                             zip_file_name, zip_file_name.replace(".zip", ".csv"),
                                                 "endereco", "TOTAL",uf, ano)
     

https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012
['ben202012_GO.zip']
ben202012_GO.zip
ben202012_GO.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012/ben202012_GO.zip


23/10/24 12:50:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012
['ben202012_MG.zip']
ben202012_MG.zip
ben202012_MG.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012/ben202012_MG.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012
['ben202012_BA.zip']
ben202012_BA.zip
ben202012_BA.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202012/ben202012_BA.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112
['ben202112_GO.zip']
ben202112_GO.zip
ben202112_GO.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112/ben202112_GO.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112
['ben202112_MG.zip']
ben202112_MG.zip
ben202112_MG.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112/ben202112_MG.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112
['ben202112_BA.zip']
ben202112_BA.zip
ben202112_BA.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202112/ben202112_BA.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212
['ben202212_GO.zip']
ben202212_GO.zip
ben202212_GO.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212/ben202212_GO.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212
['ben202212_MG.zip']
ben202212_MG.zip
ben202212_MG.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212/ben202212_MG.zip


https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212
['ben202212_BA.zip']
ben202212_BA.zip
ben202212_BA.csv
https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202212/ben202212_BA.zip


## Download das bases de dados complementáres de códigos TUSS 

### Utilizada para normalização e acesso de referencia de procedimentos realizados

In [96]:
# URL da pasta compactada
url = "https://dadosabertos.ans.gov.br/FTP/PDA/terminologia_unificada_saude_suplementar_TUSS/TUSS.zip"

# Faça o download do arquivo
response = requests.get(url)
    # Crie um objeto BytesIO para trabalhar com os dados
zip_data = io.BytesIO(response.content)
    
    # Descompacte o arquivo
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    zip_ref.extractall("/Users/matheuscosta/Untitled Folder/TUSS")



## Download de base de rede credenciada para as UF's alvo

In [145]:
# URL da pasta compactada
url = "https://dadosabertos.ans.gov.br/FTP/PDA/operadoras_e_prestadores_nao_hospitalares/operadoras_e_prestadores_nao_hospitalares.zip"

# Faça o download do arquivo
response = requests.get(url)
    # Crie um objeto BytesIO para trabalhar com os dados
zip_data = io.BytesIO(response.content)
    
    # Descompacte o arquivo
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    zip_ref.extractall("/Users/matheuscosta/Untitled Folder/prestador_amb")
    
aux = os.listdir("/Users/matheuscosta/Untitled Folder/prestador_amb")
aux = [item for item in aux if "GO" not in item]
aux = [item for item in aux if "MG" not in item]
aux = [item for item in aux if "BA" not in item]
for file in aux:
    os.remove(os.path.join("/Users/matheuscosta/Untitled Folder/prestador_amb", file))

## Os demais códigos abaixo são projetos de implementação para o consumo de rede credenciada hospitalar para as praças desejadas

In [147]:
# URL da pasta compactada
#url = "https://dadosabertos.ans.gov.br/FTP/PDA/produtos_e_prestadores_hospitalares/produtos_e_prestadores_hospitalares.zip"

# Faça o download do arquivo
#response = requests.get(url)
    # Crie um objeto BytesIO para trabalhar com os dados
#zip_data = io.BytesIO(response.content)
    
    # Descompacte o arquivo
#with zipfile.ZipFile(zip_data, 'r') as zip_ref:
#    zip_ref.extractall("/Users/matheuscosta/Untitled Folder/prestador_hosp")
    
#aux = os.listdir("/Users/matheuscosta/Untitled Folder/prestador_hosp")
#aux = [item for item in aux if "GO" not in item]
#aux = [item for item in aux if "MG" not in item]
#aux = [item for item in aux if "BA" not in item]
#for file in aux:
#    os.remove(os.path.join("/Users/matheuscosta/Untitled Folder/prestador_hosp", file))

OSError: [Errno 28] No space left on device

In [152]:
# URL da pasta compactada
#zip_file_url = "https://dadosabertos.ans.gov.br/FTP/PDA/produtos_e_prestadores_hospitalares/produtos_e_prestadores_hospitalares.zip"
#zip_file_name = "produtos_e_prestadores_hospitalares.zip"
# Faça o download do arquivo
#response = requests.get(url)
    # Crie um objeto BytesIO para trabalhar com os dados
#zip_data = io.BytesIO(response.content)
    
    # Descompacte o arquivo
#with zipfile.ZipFile(zip_data, 'r') as zip_ref:
#    zip_ref.extractall("/Users/matheuscosta/Untitled Folder/prestador_hosp")
    
#with tempfile.TemporaryDirectory() as temp_dir:
#        # Baixar o arquivo ZIP
#        response = requests.get(zip_file_url)
#        with open(f"{temp_dir}/{zip_file_name}", "wb") as zip_file:
#            zip_file.write(response.content)

        # Descompactar o arquivo ZIP
#        with zipfile.ZipFile(f"{temp_dir}/{zip_file_name}", "r") as zip_ref:
#            zip_ref.extractall(temp_dir)
#            aux = os.listdir(temp_dir)
#            prin(aux)


OSError: [Errno 28] No space left on device